In [1]:
import sagemaker
import boto3

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
# if sagemaker_session_bucket is None and sess is not None:
#     # set to default bucket if a bucket name is not given
#     sagemaker_session_bucket = sess.default_bucket()

sagemaker_session_bucket = "sentence-transformers-sagemaker-hf-150523"

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="AmazonSageMaker-ExecutionRole-20220224T111993")[
        "Role"
    ]["Arn"]

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu-xorg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/alli/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu-xorg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/alli/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu-xorg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/alli/.config/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name Admin to get Role path.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu-xorg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/alli/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::822248934593:role/service-role/AmazonSageMaker-ExecutionRole-20220224T111993
sagemaker bucket: sentence-transformers-sagemaker-hf-150523
sagemaker session region: us-west-2


In [2]:
sess.default_bucket() 

'sentence-transformers-sagemaker-hf-150523'

In [2]:
repository = "sentence-transformers/all-mpnet-base-v2"

model_id = repository.split("/")[-1]
s3_location = f"s3://{sess.default_bucket()}/custom_inference/{model_id}/model.tar.gz"

In [19]:
# !git lfs install 
# !git clone https://huggingface.co/$repository 

In [20]:
# !cp -r code/ $model_id/code/

In [21]:
# %cd $model_id
# !tar zcvf model.tar.gz *

/home/alli/Documents/GitHub/chatbot-personal-data/all-MiniLM-L6-v2
1_Pooling/
1_Pooling/config.json
code/
code/inference.py
config.json
config_sentence_transformers.json
data_config.json
modules.json
README.md


In [4]:
print(s3_location)

s3://sentence-transformers-sagemaker-hf-150523/custom_inference/all-mpnet-base-v2/model.tar.gz


In [22]:
# !aws s3 cp model.tar.gz $s3_location

upload: ./model.tar.gz to s3://sentence-transformers-sagemaker-hf-150523/custom_inference/all-MiniLM-L6-v2/model.tar.gz


In [3]:
print(s3_location)

s3://sentence-transformers-sagemaker-hf-150523/custom_inference/all-mpnet-base-v2/model.tar.gz


In [5]:
from sagemaker.huggingface.model import HuggingFaceModel


# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    model_data=s3_location,
    role=role,
    transformers_version="4.28",
    pytorch_version="2.0",
    py_version="py310",
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name="sentence-transformers-all-mpnet-base-v2",
)

# "ml.g4dn.xlarge"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu-xorg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/alli/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu-xorg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/alli/.config/sagemaker/config.yaml
----------!

In [6]:
data = {
  "inputs": "the mesmerizing performances of the leads keep the film grounded and keep the audience riveted .",
}

res = predictor.predict(data=data)
print(res)

{'vectors': [[-0.049686044454574585, -0.005454717203974724, -0.014014150016009808, 0.029759569093585014, 0.004519848618656397, 0.03545575961470604, -0.08696193993091583, -0.026570141315460205, -0.018208596855401993, 0.02694297954440117, -0.0411970391869545, -0.0014295931905508041, -0.0018213184084743261, 0.043491896241903305, -0.0030700378119945526, -0.06470834463834763, -0.018531514331698418, 0.022012315690517426, -0.06463608145713806, 0.003242586040869355, -0.01053592935204506, -0.028615236282348633, 0.01746680773794651, -0.028870346024632454, 0.008701849728822708, -0.017756929621100426, 0.03698035702109337, 0.03468147665262222, 0.043723903596401215, -0.046832188963890076, 0.002466214820742607, -0.00022224945132620633, -0.008667804300785065, 0.02073836140334606, 1.3844269233231898e-06, -0.01640620082616806, -0.017006389796733856, -0.055704060941934586, 0.010235000401735306, 0.016042856499552727, -0.012508087791502476, 0.07923313975334167, -0.037915099412202835, -0.020938875153660774,

In [7]:
predictor.delete_endpoint()

In [30]:
# import json

# sagemaker_client = boto3.client("sagemaker-runtime")

# # Define the endpoint name and payload
# endpoint_name = "sentence-transformers-all-MiniLM-L6-v2"

# payload = {
#     "inputs": "Who is Bola Ahmed Tinubu?",
# }

# # Send the request to the SageMaker endpoint
# response = sagemaker_client.invoke_endpoint(
#     EndpointName=endpoint_name,
#     ContentType="application/json",
#     Body=json.dumps(payload),
# )